# Haversine Formula vs Pythagorean Theorem

## Written by Michael George

Simple comparison of two methods to calculate the distance between two points of close proximity on a sphere:

1. Haversine formula is required over longer distances (e.g. navigation over hundreds of kilometres).
2. Pythagorean theorem can be used to produce quicker estimates over shorter distances (e.g. alpha proximity 50m).

The article "[A simple benchmark of various math operations](https://latkin.org/blog/2014/11/09/a-simple-benchmark-of-various-math-operations/)" illustrates the relative computational costs of different mathematical operations.

In [1]:
from math import pi, radians, cos, sin, asin, atan2, sqrt

import unittest

## Constants

Earth's [mean radius](https://en.wikipedia.org/wiki/Earth_radius#Published_values) was taken from Wikipedia - IUGG and IERS both give a value of 6371008.7714 metres.

In [2]:
EARTH_RADIUS = 6371009
EARTH_CIRCUM = 2 * pi * EARTH_RADIUS
DIST_PER_DEG_LAT = EARTH_CIRCUM / 360

## Unit Conversions

In [3]:
def semicirclesToDegrees(value):
    '''Convert semicircles to degrees'''

    return value * (180.0 / 2 ** 31)


def semicirclesToRadians(value):
    '''Convert semicircles to radians'''

    return radians(value * (180.0 / 2 ** 31))


def metresToKilometers(value):
    '''Convert metres to kilometers'''

    return value / 1000


def metresToMiles(value):
    '''Convert metres to kilometers'''

    return value / 1609.344

## Haversine Formula

A popular approach when calculating the distance between two points is to use the Haversine formula.

This requires the use of several math functions (e.g. 2 * sin, 2 * cos, 1 * asin, 1 * sqrt) with little opportunity to optimise.

Full details about the Haversine formula can be found on [Wikipedia](https://en.wikipedia.org/wiki/Haversine_formula).

In [4]:
def calculateDistanceRadians(latitude1r, longitude1r, latitude2r, longitude2r, radius=EARTH_RADIUS):
    '''Calculate the great-circle distance between two points on a sphere given their longitudes and latitudes'''

    # Cost comparison - operations that cannot be cached
    # Accurate Cost: Asin = 1, Sin = 2, Cos = 0, Sqrt = 1, Divide = 2, Square = 2, Multiply = 3, Subtract = 2, Add = 1
    # Estimate Cost: Asin = 0, Sin = 0, Cos = 1, Sqrt = 1, Divide = 0, Square = 2, Multiply = 1, Subtract = 2, Add = 1

    # Apply the haversine formula. n.b. Can cache the individual cos() results
    cosineCalc = cos(latitude1r) * cos(latitude2r)
    haversine = sin((latitude2r - latitude1r) / 2) ** 2 + cosineCalc * sin((longitude2r - longitude1r) / 2) ** 2

    # Alternative hathersine formula avoids cos() byt replaces it with a more "expensive" calculation
    #sineCalc = 1 - sin((latitude1 - latitude2) / 2) ** 2 - sin((latitude1 + latitude2) / 2) ** 2
    #haversine = sin((latitude2 - latitude1) / 2) ** 2 + sineCalc * sin((longitude2 - longitude1) / 2) ** 2

    distance = 2 * radius * asin(sqrt(haversine))

    # Alternative distance calculation produces the same result and was only tested out of curiosity
    #distance = 2 * radius * atan2(sqrt(haversine), sqrt(1 - haversine))

    return distance


def calculateDistanceDegrees(latitude1d, longitude1d, latitude2d, longitude2d, radius=EARTH_RADIUS):
    '''Calculate the great-circle distance between two points on a sphere given their longitudes and latitudes'''

    latitude1, longitude1, latitude2, longitude2 = map(radians, [latitude1d, longitude1d, latitude2d, longitude2d])

    return calculateDistanceRadians(latitude1, longitude1, latitude2, longitude2, radius=radius)


def calculateDistanceSemicircles(latitude1s, longitude1s, latitude2s, longitude2s, radius=EARTH_RADIUS):
    '''Calculate the great-circle distance between two points on a sphere given their longitudes and latitudes'''

    latitude1, longitude1, latitude2, longitude2 = map(semicirclesToRadians, [latitude1s, longitude1s, latitude2s, longitude2s])

    return calculateDistanceRadians(latitude1, longitude1, latitude2, longitude2, radius=radius)

## Pythagorean Theorem

A quicker way approximate the distance between two points is to map coordinates to a 2D grid and use the Pythagorean theorum.

This requires far less of the costly math functions (1 * cos, 1 * sqrt) ona frequent basis.

In [5]:
def estimateDistanceRadians(latitude1r, longitude1r, latitude2r, longitude2r, radius=EARTH_RADIUS):
    '''Estimate the Euclidean distance between two points on a sphere using Pythagorean Theorem'''

    # Cost comparison - operations that cannot be cached
    # Accurate Cost: Asin = 1, Sin = 2, Cos = 0, Sqrt = 1, Divide = 2, Square = 2, Multiply = 3, Subtract = 2, Add = 1
    # Estimate Cost: Asin = 0, Sin = 0, Cos = 1, Sqrt = 1, Divide = 0, Square = 2, Multiply = 1, Subtract = 2, Add = 1

    # Calculate distance north / south. n.b. Can cache the values of y1 + y2
    y1 = latitude1r * EARTH_RADIUS
    y2 = latitude2r * EARTH_RADIUS
    yDelta = y2 - y1

    # For the sake of completeness we need to cope with the points either side of the 180th meridian
    longDelta = abs(longitude2r - longitude1r)

    # Note: This adjustment has not been factored into the "cost" comparison above because it is so unlikely to occur!
    if longDelta > pi:
        longDelta -= 2 * pi

    # Calculate distance east / west. n.b. Can cache EARTH_RADIUS * cos(latitude2r)
    xDelta = longDelta * EARTH_RADIUS * cos(latitude2r)

    # Apply Pythagorean theorem to determine the distance
    distance = sqrt(xDelta ** 2 + yDelta ** 2)

    return distance


def estimateDistanceDegrees(latitude1d, longitude1d, latitude2d, longitude2d, radius=EARTH_RADIUS):
    '''Estimate the Euclidean distance between two points on a sphere using Pythagorean Theorem'''

    # Calculate distance north / south. n.b. Can cache the values of y1 + y2
    y1 = latitude1d * DIST_PER_DEG_LAT
    y2 = latitude2d * DIST_PER_DEG_LAT
    yDelta = y2 - y1

    # For the sake of completeness we need to cope with the points either side of the 180th meridian
    longDelta = abs(longitude2d - longitude1d)

    # Note: This adjustment has not been factored into the "cost" comparison above because it is so unlikely to occur!
    if longDelta > 180:
        longDelta -= 360

    # Calculate distance east / west. n.b. Can cache DIST_PER_DEG_LAT * cos(radians(latitude2d))
    xDelta = longDelta * DIST_PER_DEG_LAT * cos(radians(latitude2d))

    # Apply Pythagorean theorem to determine the distance
    distance = sqrt(xDelta ** 2 + yDelta ** 2)

    return distance


def estimateDistanceSemicircles(latitude1s, longitude1s, latitude2s, longitude2s, radius=EARTH_RADIUS):
    '''Estimate the Euclidean distance between two points on a sphere using Pythagorean Theorem'''

    latitude1, longitude1, latitude2, longitude2 = map(semicirclesToRadians, [latitude1s, longitude1s, latitude2s, longitude2s])

    return estimateDistanceRadians(latitude1, longitude1, latitude2, longitude2, radius=radius)

## Unit Tests

A handful of tests to ensure the Haversine and Pythagoras functions are working properly!

I used some online tools to check the expected values - e.g. [vCalc](https://www.vcalc.com/wiki/vCalc/Haversine+-+Distance)

In [6]:
class TestSemicircles(unittest.TestCase):
    '''Class to test distance calculations when specifying latitude + longitude in semicircles'''

    def testBrogAlphaProximity(self):
        '''Test alpha proximity of around 46.50 metres - Pythagoras estimate is mm accurate'''

        lat1s = 620951524
        lon1s = -6830082

        lat2s = 620954030
        lon2s = -6823068

        distance = calculateDistanceSemicircles(lat1s, lon1s, lat2s, lon2s)
        self.assertEqual(round(distance, 3), 46.496)

        distance = estimateDistanceSemicircles(lat1s, lon1s, lat2s, lon2s)
        self.assertEqual(round(distance, 3), 46.496)



class TestDegrees(unittest.TestCase):
    '''Class to test distance calculations when specifying latitude + longitude in degrees'''

    def testBrogAlphaProximity(self):
        '''Test alpha proximity of around 46.50 metres - Pythagoras estimate is mm accurate'''

        lat1d = semicirclesToDegrees(620951524)
        lon1d = semicirclesToDegrees(-6830082)

        lat2d = semicirclesToDegrees(620954030)
        lon2d = semicirclesToDegrees(-6823068)

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(distance, 3), 46.496)

        distance = estimateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(distance, 3), 46.496)


    def testStevenageWare(self):
        '''Test Stevenage (Herts) to Ware (Herts) which is around 15.5 km. Pythagoras estimate is accurate to within 9 cm'''

        lat1d = 51.9038
        lon1d = -0.1966

        lat2d = 51.8104
        lon2d = -0.0282

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 2), 15.54)

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 15.544)

        distance = estimateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 15.553)


    def testStevenageWareAlt(self):
        '''Test Stevenage (Herts) to Ware (Herts) equivalent but on the other side of the globe, also accurate to within 9 cm'''

        lat1d = 51.9038
        lon1d = -179.1966

        lat2d = 51.8104
        lon2d = -179.0282

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 2), 15.54)

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 15.544)

        distance = estimateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 15.553)


    def testHighLatitude(self):
        '''Test high latitude travelling ~38.62 km due east. Pythagoras estimate is accurate to 2 metres (+0.005%)'''

        lat1d = 80
        lon1d = 0

        lat2d = 80
        lon2d = 2

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 2), 38.62)

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 38.616)

        distance = estimateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 38.618)


    def test180thMeridian(self):
        '''Test handling of the 180th Meridian at a high latitude travelling  ~38.62 km due west, also accurate to 2 metres'''

        lat1d = 80
        lon1d = -179

        lat2d = 80
        lon2d = 179

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 2), 38.62)

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 38.616)

        distance = estimateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 38.618)


    def testStevenagePortland(self):
        '''Test Stevenage (Herts) to Portland (Dorset) which is ~216.8 km. Pythagoras estimate is accurate to 1.7 km (+0.8%)'''

        lat1d = 51.9038
        lon1d = -0.1966

        lat2d = 50.5475
        lon2d = -2.4343

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 2), 216.84)

        distance = calculateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 216.837)

        distance = estimateDistanceDegrees(lat1d, lon1d, lat2d, lon2d)
        self.assertEqual(round(metresToKilometers(distance), 3), 218.503)

In [7]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.003s

OK
